In [ ]:
import pandas as pd
import numpy as np
import chess
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# Load the chess player dataset
data_path = r"C:\Users\hiddensardar\Downloads\ches.csv"
df = pd.read_csv(data_path)

# Function to convert descriptive moves to UCI format
def convert_to_uci(move, board):
    # Try converting the move using the chess library's parsing functions
    try:
        legal_moves = list(board.legal_moves)
        for legal_move in legal_moves:
            if board.san(legal_move) == move:  # Match the move in SAN (e.g., 'e4') format
                return legal_move.uci()
        return None
    except:
        return None

# Function to convert board position to features
def board_to_features(board):
    piece_mapping = {
        'P': 1, 'N': 2, 'B': 3, 'R': 4, 'Q': 5, 'K': 6,
        'p': -1, 'n': -2, 'b': -3, 'r': -4, 'q': -5, 'k': -6,
        '.': 0
    }
    features = []
    for row in range(8):
        for col in range(8):
            piece = board.piece_at(chess.square(col, row))
            features.append(piece_mapping[piece.symbol()] if piece else 0)
    return np.array(features)

# Prepare data for model training
X = []  # Features
y = []  # Labels (target moves)

# Iterate through the dataset to create features and labels
for index, row in df.iterrows():
    board = chess.Board()
    moves = row['WhiteMoves'] if row['White'] == 'Donchenko, Alexander' else row['BlackMoves']
    moves = eval(moves)  # Convert string to list format

    for move in moves:
        # Convert to UCI format if needed
        uci_move = convert_to_uci(move, board)
        if uci_move is None:
            continue  # Skip invalid or non-convertible moves

        # Create features from the current board
        features = board_to_features(board)
        X.append(features)
        y.append(uci_move)  # Using UCI move format
        
        # Make the move on the board
        board.push_uci(uci_move)

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Save the trained model to a file
model_save_path = r"C:\Users\hiddensardar\Downloads\chess_model.pkl"
joblib.dump(model, model_save_path)

# Confirm that the model is saved
print(f"Model saved to {model_save_path}")
